Experiment to use lora to make a lying model. Here we think of Lora as a probe, as it acts in a very similar way - modifying the residual stream.

Then the hope is it will assist at lie detecting and generalize to unseen dataset

- https://github.dev/JD-P/minihf/blob/b54075c34ef88d9550e37fdf709e78e5a68787c4/lora_tune.py
- https://github.com/jonkrohn/NLP-with-LLMs

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]= "1"


In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

plt.style.use("ggplot")

from typing import Optional, List, Dict, Union
from jaxtyping import Float
from torch import Tensor

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
from einops import rearrange

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, LoftQConfig, IA3Config

import datasets
from datasets import Dataset

from loguru import logger

logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


# # quiet please
torch.set_float32_matmul_precision("medium")
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings(
#     "ignore", ".*sampler has shuffling enabled, it is strongly recommended that.*"
# )
# warnings.filterwarnings("ignore", ".*has been removed as a dependency of.*")


/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load my code
%load_ext autoreload
%autoreload 2

import lightning.pytorch as pl
from src.datasets.dm import DeceptionDataModule
from src.models.pl_lora_ft import AtapterFinetuner

from src.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset
from src.models.load import load_model


## Parameters


In [4]:
# params
max_epochs = 1
device = "cuda:0"

cfg = ExtractConfig(
    batch_size=3,
    max_examples=(400, 150),
    intervention_fit_examples=60,
)


## Load model

In [5]:
model, tokenizer = load_model(
    cfg.model,
    device=device,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# TODO I would like to only have biases, but for now lets just try a very small intervention on the last parts of a layer...
peft_config = LoraConfig(
    target_modules=[
        "out_proj",
        "mlp.fc2",
    ],  # only the layers that go directly to the residual
    # bias="lora_only",
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=1,
    lora_dropout=0.0,
)


# peft_config = IA3Config(
#     task_type=TaskType.SEQ_CLS, target_modules=[ "out_proj",
#         "mlp.fc2",], feedforward_modules=["out_proj", "mlp.fc2",]
# )
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 1,376,256 || all params: 1,419,646,976 || trainable%: 0.09694353760240743


In [7]:
N = sum(cfg.max_examples)
ds_name = "amazon_polarity"
ds_tokens = load_preproc_dataset(
    ds_name,
    tokenizer,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
).with_format("torch")


2023-12-21 12:37:44.140 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:364 - median token length: 302.0 for amazon_polarity. max_length=777
2023-12-21T12:37:44.140659+0800 INFO median token length: 302.0 for amazon_polarity. max_length=777
2023-12-21 12:37:44.142 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:368 - truncation rate: 0.00% on amazon_polarity
2023-12-21T12:37:44.142693+0800 INFO truncation rate: 0.00% on amazon_polarity
Filter: 100%|██████████| 1652/1652 [00:00<00:00, 2579.22 examples/s]
2023-12-21 12:37:44.797 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:377 - num_rows (after filtering out truncated rows) 1652=>1652
2023-12-21T12:37:44.797039+0800 INFO num_rows (after filtering out truncated rows) 1652=>1652


## Train

In [9]:
dm = DeceptionDataModule(ds_tokens, batch_size=cfg.batch_size)
dm


In [10]:
dl_train = dm.train_dataloader()
dl_val = dm.val_dataloader()


In [11]:
b = next(iter(dl_train))
print(b.keys(), b["input_ids"].shape)
c_in = b["input_ids"].shape[1]
c_in


dict_keys(['ds_string', 'example_i', 'answer', 'messages', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'question', 'input_ids', 'attention_mask', 'truncated', 'length', 'prompt_truncated', 'choice_ids']) torch.Size([3, 777])


777

In [12]:
net = AtapterFinetuner(
    model, tokenizer, lr=5e-5, weight_decay=0, total_steps=len(dl_train) * max_epochs
)

print(c_in)
# net.model.enable_adapters()


777


In [13]:
# debug
with torch.no_grad():
    o = net.training_step(b, None)
o


/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.venv/lib/python3.11/site-packages/lightning/pytorch/core/module.py:420: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(1.6051)

In [14]:
# debug
with torch.no_grad():
    o = net.predict_step(b, None)
# o


In [15]:
# we want to init lightning early, so it inits accelerate
trainer1 = pl.Trainer(
    # precision="16-true", # works?
    # precision="16-mixed",
    # precision="b16-mixed",
    gradient_clip_val=20,
    # accelerator="auto",
    # devices="1",
    # accelerator="gpu",
    # devices=[0],
    # accumulate_grad_batches=2,
    max_epochs=max_epochs,
    log_every_n_steps=1,
    # enable_progress_bar=False,
    enable_model_summary=False,
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoar

In [16]:
trainer1.accelerator


In [17]:
trainer1.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val);


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 0: 100%|██████████| 92/92 [01:50<00:00,  0.83it/s, v_num=105]        

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 92/92 [01:51<00:00,  0.82it/s, v_num=105]


In [18]:
checkpoint_path = Path(trainer1.log_dir)/'final'
model.save_pretrained(checkpoint_path)


In [19]:
from src.helpers.lightning import read_metrics_csv

pd.read_csv(trainer1.logger.experiment.metrics_file_path).bfill().ffill()


,step,epoch,train/loss_step,val/loss_step,val/loss_epoch,train/loss_epoch
0,0,0.0,5.056456,5.617131,3.387286,3.260047
1,1,0.0,4.220459,5.617131,3.387286,3.260047
2,2,0.0,2.165426,5.617131,3.387286,3.260047
3,3,0.0,2.463885,5.617131,3.387286,3.260047
4,4,0.0,4.388612,5.617131,3.387286,3.260047
...,...,...,...,...,...,...
135,43,0.0,0.860126,0.701600,3.387286,3.260047
136,44,0.0,0.860126,0.948741,3.387286,3.260047
137,45,0.0,0.860126,1.151066,3.387286,3.260047
138,91,0.0,0.860126,1.151066,3.387286,3.260047


In [20]:
# df_hist_epoch, df_hist_step = read_metrics_csv(trainer1.logger.experiment.metrics_file_path)
# # for key in ["loss"]:
# #     df_hist_epoch[[c for c in df_hist_epoch.columns if key in c]].plot(logy=True)

# df_hist_step = df_hist_step.dropna()
# df_hist_step.plot()
# df_hist_step


## Generate


In [21]:
# get a row
bi = cfg.intervention_fit_examples + 4
inputs = ds_tokens.with_format("torch")[bi]


In [22]:
from IPython.display import display, HTML

# generate
# https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/text_generation#transformers.GenerationConfig


@torch.no_grad()
def gen(model):
    s = model.generate(
        input_ids=inputs["input_ids"][None, :].to(model.device),
        attention_mask=inputs["attention_mask"][None, :].to(model.device),
        use_cache=False,
        max_new_tokens=100,
        min_new_tokens=100,
        do_sample=False,
        early_stopping=False,
    )
    input_l = inputs["input_ids"].shape[0]
    old = tokenizer.decode(
        s[0, :input_l], clean_up_tokenization_spaces=False, skip_special_tokens=False
    )
    new = tokenizer.decode(
        s[0, input_l:], clean_up_tokenization_spaces=False, skip_special_tokens=False
    )
    display(HTML(f"<pre>{old}</pre><b><pre>{new}</pre></b>"))


In [23]:
# debug
with torch.no_grad():
    o = net.predict_step(b, None)
# o


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [24]:
# debug
with torch.no_grad():
    o = net.training_step(b, None)
o


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [25]:
# # with model.disable_adapters():
# with model.disable_adapter():
#     gen(model)

# gen(model)

# # RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'


In [38]:
model, tokenizer = model, tokenizer = load_model(
    cfg.model,
    device=device,
    adaptor_path=checkpoint_path,
    # bnb=False
)
net = AtapterFinetuner(
    model, tokenizer, lr=5e-5, weight_decay=0, total_steps=len(dl_train) * max_epochs
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
# # debug
# with torch.no_grad():
#     o = net.predict_step(b, None)
# o


In [ ]:
# with model.disable_adapters():
with model.disable_adapter():
    gen(model)

gen(model)


# Test

In [78]:
N = sum(cfg.max_examples)
ds_name = "imdb"
ds_tokens2 = load_preproc_dataset(
    ds_name,
    tokenizer,
    N=N // 4,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
).with_format("torch")


2023-12-21 12:47:29.716 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:364 - median token length: 593.0 for imdb. max_length=777
2023-12-21T12:47:29.716359+0800 INFO median token length: 593.0 for imdb. max_length=777
2023-12-21 12:47:29.717 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:368 - truncation rate: 16.75% on imdb
2023-12-21T12:47:29.717720+0800 INFO truncation rate: 16.75% on imdb
Filter: 100%|██████████| 343/343 [00:00<00:00, 2447.47 examples/s]
2023-12-21 12:47:29.873 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:377 - num_rows (after filtering out truncated rows) 412=>343
2023-12-21T12:47:29.873955+0800 INFO num_rows (after filtering out truncated rows) 412=>343


In [79]:
dm = DeceptionDataModule(ds_tokens, batch_size=cfg.batch_size * 2)
dl_train2 = dm.train_dataloader()
dl_val2 = dm.val_dataloader()
dl_test2 = dm.test_dataloader()


In [80]:
dl_oos2 = DataLoader(
    ds_tokens2, batch_size=cfg.batch_size * 2, drop_last=False, shuffle=False
)


In [81]:
# def get_acc_subset(df, query, verbose=True):
#     if query:
#         df = df.query(query)
#     acc = (df["probe_pred"] == df["y"]).mean()
#     if verbose:
#         print(f"acc={acc:2.2%},\tn={len(df)},\t[{query}] ")
#     return acc


# def calc_metrics(dm, trainer, net, use_val=False, verbose=True):
#     dl_test = dm.test_dataloader()
#     rt = trainer.predict(net, dataloaders=dl_test)
#     y_test_pred = np.concatenate(rt)
#     splits = dm.splits["test"]
#     df_test = dm.df.iloc[splits[0] : splits[1]].copy()
#     df_test["probe_pred"] = y_test_pred > 0.0

#     if use_val:
#         dl_val = dm.val_dataloader()
#         rv = trainer.predict(net, dataloaders=dl_val)
#         y_val_pred = np.concatenate(rv)
#         splits = dm.splits["val"]
#         df_val = dm.df.iloc[splits[0] : splits[1]].copy()
#         df_val["probe_pred"] = y_val_pred > 0.0

#         df_test = pd.concat([df_val, df_test])

#     if verbose:
#         print("probe results on subsets of the data")
#     acc = get_acc_subset(df_test, "", verbose=verbose)
#     get_acc_subset(
#         df_test, "instructed_to_lie==True", verbose=verbose
#     )  # it was ph told to lie
#     get_acc_subset(
#         df_test, "instructed_to_lie==False", verbose=verbose
#     )  # it was told not to lie
#     get_acc_subset(
#         df_test, "llm_ans==label_true", verbose=verbose
#     )  # the llm gave the true ans
#     get_acc_subset(
#         df_test, "llm_ans==label_instructed", verbose=verbose
#     )  # the llm gave the desired ans
#     acc_lie_lie = get_acc_subset(
#         df_test, "instructed_to_lie==True & llm_ans==label_instructed", verbose=verbose
#     )  # it was told to lie, and it did lie
#     acc_lie_truth = get_acc_subset(
#         df_test, "instructed_to_lie==True & llm_ans!=label_instructed", verbose=verbose
#     )

#     a = get_acc_subset(
#         df_test, "instructed_to_lie==False & llm_ans==label_instructed", verbose=False
#     )
#     b = get_acc_subset(
#         df_test, "instructed_to_lie==False & llm_ans!=label_instructed", verbose=False
#     )
#     c = get_acc_subset(
#         df_test, "instructed_to_lie==True & llm_ans==label_instructed", verbose=False
#     )
#     d = get_acc_subset(
#         df_test, "instructed_to_lie==True & llm_ans!=label_instructed", verbose=False
#     )
#     d1 = pd.DataFrame(
#         [[a, b], [c, d]],
#         index=["instructed_to_lie==False", "instructed_to_lie==True"],
#         columns=["llm_ans==label_instructed", "llm_ans!=label_instructed"],
#     )
#     d1 = pd.DataFrame(
#         [[a, b], [c, d]],
#         index=["tell a truth", "tell a lie"],
#         columns=["did", "didn't"],
#     )
#     d1.index.name = "instructed to"
#     d1.columns.name = "llm gave"
#     print("probe accuracy for quadrants")
#     display(d1.round(2))

#     if verbose:
#         print(f"⭐PRIMARY METRIC⭐ acc={acc:2.2%} from probe")
#         print(f"⭐SECONDARY METRIC⭐ acc_lie_lie={acc_lie_lie:2.2%} from probe")
#     return dict(acc=acc, acc_lie_lie=acc_lie_lie, acc_lie_truth=acc_lie_truth)


In [82]:
# import re


# def transform_dl_k(k: str) -> str:
#     p = re.match(r"test\/(.+)\/dataloader_idx_\d", k)
#     return p.group(1) if p else k


# def rename(rs, ks=["train", "val", "test"]):
#     rs = {
#         ks[i]: {transform_dl_k(k): v for k, v in rs[i].items()} for i in range(len(ks))
#     }
#     return rs


In [83]:
# model.device


In [84]:
# rs = trainer1.test(
#     net,
#     dataloaders=[
#         # dl_train2, dl_val2,
#         dl_test2,
#         dl_oos2,
#     ],
# )
# rs = rename(rs, ["train", "val", "test", "oos"])
# rs[0]


# Predict

Here we want to see if we can do a probe on the hidden states to see if it's lying...


In [129]:
accelerator = trainer1.accelerator
model, tokenizer = model, tokenizer = load_model(
    cfg.model,
    device=device,
    adaptor_path=checkpoint_path,
)
net = AtapterFinetuner(
    model, tokenizer, lr=5e-5, weight_decay=0, total_steps=len(dl_train) * max_epochs
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [130]:
rv = trainer1.predict(net, dataloaders=dl_val2)
# convert from List[Dict[Tensor] to Dict[Tensor]
rv = Dataset.from_dict({k: torch.concat([rr[k] for rr in rv]) for k in rv[0].keys()})
rv


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0:   0%|          | 0/6 [00:15<?, ?it/s]


Predicting: |          | 6/? [00:07<00:00,  0.78it/s]


Dataset({
    features: ['end_hidden_states_base', 'end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'instructed_to_lie_base', 'end_hidden_states_adapt', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'instructed_to_lie_adapt'],
    num_rows: 34
})

6

In [115]:
len(rv), rv[0].keys(), rv[0]['end_logits_base'].shape


(6,
 dict_keys(['end_hidden_states_base', 'end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'instructed_to_lie_base', 'end_hidden_states_adapt', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'instructed_to_lie_adapt']),
 torch.Size([6, 51200]))

In [113]:
Dataset.from_list(rv)


Dataset({
    features: ['end_hidden_states_base', 'end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'instructed_to_lie_base', 'end_hidden_states_adapt', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'instructed_to_lie_adapt'],
    num_rows: 6
})

In [111]:
len(rv), rv[0].keys()


(6,
 dict_keys(['end_hidden_states_base', 'end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'instructed_to_lie_base', 'end_hidden_states_adapt', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'instructed_to_lie_adapt']))

In [ ]:
testval_metrics = calc_metrics(dm, trainer1, net, use_val=True)

# rs['test'] = {**rs['test'], **test_metrics}
rs["test"]["acc_lie_lie"] = testval_metrics["acc_lie_lie"]
rs["testval_metrics"] = rs["test"]
